In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

tf.get_logger().setLevel('ERROR')

2023-03-20 21:41:48.068898: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 21:41:48.474650: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-20 21:41:48.474705: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-20 21:41:51.183535: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7:

In [3]:
df = pd.read_csv("../data/processed_its_top_10.csv", nrows = 5000)
df["description"] = df["description"].fillna("")
X = df["description"].fillna("None").to_numpy()
le = LabelEncoder()
y = le.fit_transform(df["service_offering"])
num_classes = len(le.classes_)
# Use the following line if you plan to use CategoricalCrossentropy as the loss function
# y = tf.keras.utils.to_categorical(y, num_classes=num_classes)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [19]:
df.groupby(df["service_offering"])["description"].count() / df.shape[0]

service_offering
Active Directory - ITS                   0.0890
Canvas LMS                               0.0174
Enterprise Software Licensing Support    0.0320
Hardware Endpoint Support                0.0220
MultiFactor Authentication               0.1138
Non ITS Support                          0.0614
Office 365                               0.0296
Online Directory                         0.0316
Other                                    0.4848
Single Sign-on                           0.0220
Spam / Duplicate Case                    0.0964
Name: description, dtype: float64

In [23]:
new_model = tf.keras.models.load_model('saved_model/my_model', compile=False)
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [26]:
test = ["My Duo Mobile app is not working. It says that the account is disabled. Please contact the Service Desk."]
pred = np.argmax(new_model.predict(x = test))
le.inverse_transform([pred])

1/1 [==============================] - 1s 657ms/step


array(['MultiFactor Authentication'], dtype=object)

In [27]:
test = ["My account is compromised, I need to change my password"]
pred = np.argmax(new_model.predict(x = test))
le.inverse_transform([pred])

1/1 [==============================] - 0s 82ms/step


array(['Active Directory - ITS'], dtype=object)

In [28]:
test = ["Wifi at Sixth college is not working. All of my friends are reporting a problem"]
pred = np.argmax(new_model.predict(x = test))
le.inverse_transform([pred])

1/1 [==============================] - 0s 82ms/step


array(['Other'], dtype=object)

In [29]:
test = ["My VPN is not working. It is stuck on trying to connect when I type in my password"]
pred = np.argmax(new_model.predict(x = test))
le.inverse_transform([pred])

1/1 [==============================] - 0s 74ms/step


array(['Other'], dtype=object)

In [30]:
test = ["I am not receiving any emails in my Outlook exchange. The last email I got was 2 months ago. HELP!!!"]
pred = np.argmax(new_model.predict(x = test))
le.inverse_transform([pred])

1/1 [==============================] - 0s 78ms/step


array(['Other'], dtype=object)

In [31]:
test = ["I need to register my Microsoft Exchange account. I can't receive any emails as of right now."]
pred = np.argmax(new_model.predict(x = test))
le.inverse_transform([pred])

1/1 [==============================] - 0s 77ms/step


array(['Active Directory - ITS'], dtype=object)

In [32]:
test = ["I just registered for Math 189 yesterday and it is not showing up as a course on my Canvas page. Please help"]
pred = np.argmax(new_model.predict(x = test))
le.inverse_transform([pred])

1/1 [==============================] - 0s 77ms/step


array(['Canvas LMS'], dtype=object)

In [34]:
test = ["My docking station is broken, the monitors connected to it does not work"]
pred = np.argmax(new_model.predict(x = test))
le.inverse_transform([pred])

1/1 [==============================] - 0s 79ms/step


array(['Other'], dtype=object)

In [35]:
test = ["My laptop is not turning on and I have work today. Please help ASAP"]
pred = np.argmax(new_model.predict(x = test))
le.inverse_transform([pred])

1/1 [==============================] - 0s 95ms/step


array(['Hardware Endpoint Support'], dtype=object)

In [36]:
test = ["My health email is not receiving any emails, what is going on?"]
pred = np.argmax(new_model.predict(x = test))
le.inverse_transform([pred])

1/1 [==============================] - 0s 73ms/step


array(['Other'], dtype=object)

In [37]:
test = ["I need Adobe Acrobat for work, how can I get that on my account?"]
pred = np.argmax(new_model.predict(x = test))
le.inverse_transform([pred])

1/1 [==============================] - 0s 81ms/step


array(['Enterprise Software Licensing Support'], dtype=object)